In [12]:
%cd /h/pwilson/projects/fun-rnn

/ssd003/home/pwilson/projects/fun-rnn


In [18]:
from train import TrainableGPTModel
import json 
import torch
from torchzero.utils.tokenizer import Tokenizer
from dataset import TextFileDataset

with open('vocab_1024.json') as f: 
    vocab = json.load(f)

d = TextFileDataset('data/bible.txt', 100000)
dl = torch.utils.data.DataLoader(d, batch_size=1, shuffle=True)

tokenizer = Tokenizer(vocab)
tokenizer.add_token('<PAD>')
tokenizer.add_token('<START>')
model = TrainableGPTModel(4, 4, 512, 764, 0.1, tokenizer, max_len=20000).to('cuda')

In [19]:
from train import prepare_batch
batch = next(iter(dl))

In [15]:
from train import prepare_batch
batch = next(iter(dl))

X = prepare_batch(batch, tokenizer, max_len=500)
X = X.to('cuda')

targets = X[:, 1:]  # shift inputs forward

truncating
500
torch.Size([1, 53151])
torch.Size([1, 500])


In [6]:
X.shape

torch.Size([64, 500])

In [8]:

X = model.embeddings(X)
B, N, D = X.shape
mask = torch.tril(torch.ones(N, N)).repeat(B, model.n_heads, 1, 1)
X, _ = model.transformer(X, mask)
X = model.classifier(X)

X = X[:, :-1, :]  # truncate outputs

In [9]:
X.shape

torch.Size([64, 499, 1027])

In [20]:
%%prun 
model.step(batch)

truncating
20000
torch.Size([1, 54205])
torch.Size([1, 20000])


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.96 GiB (GPU 0; 14.76 GiB total capacity; 9.80 GiB already allocated; 2.41 GiB free; 10.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
from train import train_epoch

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [6]:
train_epoch(dl, model, tokenizer, optimizer)

KeyboardInterrupt: 

In [9]:
%%prun
model.step(batch)

         15626 function calls (15534 primitive calls) in 0.181 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.163    0.041    0.167    0.042 transformer.py:52(forward)
        4    0.005    0.001    0.007    0.002 tokenizer.py:51(encode)
       25    0.002    0.000    0.002    0.000 {built-in method torch._C._nn.linear}
        1    0.002    0.002    0.002    0.002 {method 'repeat' of 'torch._C._TensorBase' objects}
8472/8440    0.001    0.000    0.001    0.000 {built-in method builtins.len}
     6211    0.001    0.000    0.001    0.000 {method 'append' of 'list' objects}
       34    0.001    0.000    0.001    0.000 {method 'reshape' of 'torch._C._TensorBase' objects}
        1    0.000    0.000    0.000    0.000 {built-in method torch.tensor}
        1    0.000    0.000    0.170    0.170 transformer.py:125(forward)
        1    0.000    0.000    0.181    0.181 <string>:1(<module>)
        8    0.000    0.0